In [32]:
import os
import torch
from ultralytics import YOLO
import gc

def clear_memory():
    """Clear GPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def optimize_gpu_memory():
    """Optimize GPU memory settings"""
    if torch.cuda.is_available():
        # Set memory allocation strategy
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
        
        # Clear cache
        torch.cuda.empty_cache()
        
        # Get GPU info
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"🖥️  GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Total GPU Memory: {gpu_memory:.1f} GB")
        
        # Calculate optimal batch size for different GPU memory sizes
        if gpu_memory <= 4:
            recommended_batch = 2
            recommended_workers = 2
        elif gpu_memory <= 6:
            recommended_batch = 4
            recommended_workers = 4
        elif gpu_memory <= 8:
            recommended_batch = 8
            recommended_workers = 6
        else:
            recommended_batch = 16
            recommended_workers = 8
            
        return recommended_batch, recommended_workers
    else:
        return 4, 2  # CPU fallback

def train_memory_optimized():
    print("🔴 Memory-Optimized YOLOv8 Red License Plate Training")
    print("=" * 60)
    
    # Clear memory first
    clear_memory()
    
    # Optimize memory settings
    batch_size, workers = optimize_gpu_memory()
    
    # Dataset path
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Use smaller model for limited GPU memory
    print("🤖 Using YOLOv8n (Nano) for memory efficiency...")
    model = YOLO('yolov8n.pt')
    
    # Memory-optimized training configuration
    train_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 100,
        'imgsz': 416,
        'batch': batch_size,
        'name': 'red_plate_memory_opt',
        'patience': 30,
        'save': True,
        'plots': True,
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': workers,
        'project': models_dir,  # Changed to save in dataset/models
        'exist_ok': True,
        
        # Memory optimization
        'amp': True,
        'fraction': 0.9,
        'cache': False,
        'rect': True,
        'resume': False,
        'single_cls': False,
        
        # Learning rate parameters
        'lr0': 0.005,
        'lrf': 0.1,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 2,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        
        # Loss weights
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
        
        # Data augmentation parameters
        'hsv_h': 0.01,
        'hsv_s': 0.5,
        'hsv_v': 0.3,
        'degrees': 0.0,
        'translate': 0.05,
        'scale': 0.3,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 0.5,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    print("📋 Memory-Optimized Configuration:")
    print(f"   Image size: {train_config['imgsz']}")
    print(f"   Batch size: {train_config['batch']}")
    print(f"   Workers: {train_config['workers']}")
    print(f"   AMP enabled: {train_config['amp']}")
    print(f"   Models will be saved to: {models_dir}")
    print("=" * 60)
    
    try:
        # Clear memory before training
        clear_memory()
        
        # Start training
        print("🚀 Starting memory-optimized training...")
        results = model.train(**train_config)
        
        print("🎉 Training completed successfully!")
        
        # Get the actual save directory
        save_dir = os.path.join(models_dir, 'red_plate_memory_opt')
        print(f"💾 Best model saved at: {save_dir}/weights/best.pt")
        
        # Clear memory after training
        clear_memory()
        
        # Quick validation
        print("\n🔍 Running validation...")
        metrics = model.val()
        
        print("📊 Validation Results:")
        if hasattr(metrics, 'box'):
            print(f"   mAP50: {metrics.box.map50:.4f}")
            print(f"   mAP50-95: {metrics.box.map:.4f}")
            print(f"   Precision: {metrics.box.mp:.4f}")
            print(f"   Recall: {metrics.box.mr:.4f}")
        else:
            print("   Validation metrics computed successfully")
        
        return save_dir
        
    except torch.cuda.OutOfMemoryError:
        print("❌ Still out of memory! Trying CPU training...")
        clear_memory()
        return train_on_cpu()
    except Exception as e:
        print(f"❌ Training failed: {str(e)}")
        return None

def train_on_cpu():
    """Fallback to CPU training"""
    print("🖥️  Fallback: Training on CPU...")
    
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    model = YOLO('yolov8n.pt')
    
    # CPU-optimized configuration
    cpu_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 50,
        'imgsz': 320,
        'batch': 4,
        'name': 'red_plate_cpu',
        'patience': 20,
        'save': True,
        'plots': True,
        'device': 'cpu',
        'workers': 2,
        'project': models_dir,  # Changed to save in dataset/models
        'exist_ok': True,
        
        # Minimal augmentation
        'hsv_h': 0.01,
        'hsv_s': 0.3,
        'hsv_v': 0.2,
        'degrees': 0.0,
        'translate': 0.02,
        'scale': 0.2,
        'fliplr': 0.5,
        'mosaic': 0.3,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    try:
        results = model.train(**cpu_config)
        print("🎉 CPU training completed!")
        
        # Get save directory
        save_dir = os.path.join(models_dir, 'red_plate_cpu')
        return save_dir
        
    except Exception as e:
        print(f"❌ CPU training also failed: {str(e)}")
        return None

def ultra_minimal_training():
    """Ultra minimal config for very limited resources"""
    print("🔬 Ultra Minimal Training for Limited Resources")
    
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Clear all memory
    clear_memory()
    
    model = YOLO('yolov8n.pt')
    
    minimal_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 30,
        'imgsz': 256,
        'batch': 1,
        'name': 'red_plate_minimal',
        'patience': 15,
        'save': True,
        'plots': False,
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': 0,
        'project': models_dir,  # Changed to save in dataset/models
        'exist_ok': True,
        'cache': False,
        'amp': True,
        'fraction': 0.8,
        
        # No augmentation
        'hsv_h': 0.0,
        'hsv_s': 0.0,
        'hsv_v': 0.0,
        'degrees': 0.0,
        'translate': 0.0,
        'scale': 0.0,
        'fliplr': 0.0,
        'mosaic': 0.0,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    print("⚡ Ultra minimal configuration:")
    print(f"   Image size: {minimal_config['imgsz']}")
    print(f"   Batch size: {minimal_config['batch']}")
    print(f"   Epochs: {minimal_config['epochs']}")
    print(f"   Models will be saved to: {models_dir}")
    
    try:
        results = model.train(**minimal_config)
        print("🎉 Ultra minimal training completed!")
        
        # Get save directory
        save_dir = os.path.join(models_dir, 'red_plate_minimal')
        return save_dir
        
    except Exception as e:
        print(f"❌ Even minimal training failed: {str(e)}")
        return None

def download_pretrained_models(dataset_path):
    """Download and save YOLO pretrained models to dataset directory"""
    models_dir = os.path.join(dataset_path, "pretrained_models")
    os.makedirs(models_dir, exist_ok=True)
    
    print("📥 Downloading pretrained models...")
    
    # List of YOLO models to download
    models = ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt']
    
    for model_name in models:
        try:
            print(f"   Downloading {model_name}...")
            model = YOLO(model_name)
            
            # Save model to our directory
            model_path = os.path.join(models_dir, model_name)
            torch.save(model.model.state_dict(), model_path.replace('.pt', '_weights.pt'))
            print(f"   ✅ {model_name} saved to {models_dir}")
            
        except Exception as e:
            print(f"   ❌ Failed to download {model_name}: {str(e)}")
    
    return models_dir

if __name__ == "__main__":
    # Set environment variable for memory optimization
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Dataset path
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset"
    
    print("Choose training mode:")
    print("1. Memory Optimized (Recommended)")
    print("2. CPU Training")
    print("3. Ultra Minimal")
    print("4. Download Pretrained Models Only")
    
    choice = input("Enter choice (1/2/3/4) [default: 1]: ").strip()
    
    if choice == "4":
        models_dir = download_pretrained_models(dataset_path)
        print(f"\n✅ Pretrained models downloaded to: {models_dir}")
    elif choice == "2":
        save_dir = train_on_cpu()
    elif choice == "3":
        save_dir = ultra_minimal_training()
    else:
        save_dir = train_memory_optimized()
    
    if choice != "4":
        if save_dir:
            print(f"\n✅ Training successful! Model saved at: {save_dir}")
            print(f"📁 Best weights: {save_dir}/weights/best.pt")
            print(f"📁 Last weights: {save_dir}/weights/last.pt")
            print(f"📁 Training results: {save_dir}")
            
            # Show directory structure
            print(f"\n📂 Directory structure:")
            print(f"   {dataset_path}/")
            print(f"   ├── models/")
            print(f"   │   └── {os.path.basename(save_dir)}/")
            print(f"   │       ├── weights/")
            print(f"   │       │   ├── best.pt")
            print(f"   │       │   └── last.pt")
            print(f"   │       ├── results.png")
            print(f"   │       └── confusion_matrix.png")
            print(f"   └── data.yaml")
            
        else:
            print("\n❌ All training attempts failed. Consider:")
            print("   - Using a machine with more GPU memory")
            print("   - Training on Google Colab with GPU")
            print("   - Using cloud training services")

Choose training mode:
1. Memory Optimized (Recommended)
2. CPU Training
3. Ultra Minimal
4. Download Pretrained Models Only


Enter choice (1/2/3/4) [default: 1]:  1


🔴 Memory-Optimized YOLOv8 Red License Plate Training
🖥️  GPU: NVIDIA GeForce RTX 3050 Laptop GPU
💾 Total GPU Memory: 4.0 GB
🤖 Using YOLOv8n (Nano) for memory efficiency...


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 11.3MB/s]


📋 Memory-Optimized Configuration:
   Image size: 416
   Batch size: 2
   Workers: 2
   AMP enabled: True
   Models will be saved to: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models
🚀 Starting memory-optimized training...
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/minhk/Downloads/code thu/AI/NguyenLam/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.9, freeze=None, half=False, hsv_h=0.01, hsv_s=0.5, hsv_v=0.3, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005

...


96 epochs completed in 8.145 hours.
Optimizer stripped from C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt\weights\last.pt, 6.2MB
Optimizer stripped from C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt, 6.2MB

Validating C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt...
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 512/512 [00:


                   all       2048       2195      0.983      0.901      0.954      0.647
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt
🎉 Training completed successfully!
💾 Best model saved at: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt/weights/best.pt

🔍 Running validation...
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 34.813.6 MB/s, size: 20.2 KB)


val: Scanning C:\Users\minhk\Downloads\code thuê\AI\NguyenLam\dataset\valid\labels.cache... 2048 images, 3 backgrounds,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1024/1024 [0


                   all       2048       2195      0.984      0.901      0.955      0.647
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Users\minhk\Downloads\code thu\AI\NguyenLam\dataset\models\red_plate_memory_opt
📊 Validation Results:
   mAP50: 0.9548
   mAP50-95: 0.6471
   Precision: 0.9836
   Recall: 0.9007

✅ Training successful! Model saved at: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt
📁 Best weights: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt/weights/best.pt
📁 Last weights: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt/weights/last.pt
📁 Training results: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt

📂 Directory structure:
   C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset/
   ├── models/
   │   └── red_plate_memory_opt/
   │       ├── weights/
   │       │   

In [34]:
import os
import torch
from ultralytics import YOLO
import gc

def clear_memory():
    """Clear GPU memory"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def optimize_gpu_memory():
    """Optimize GPU memory settings"""
    if torch.cuda.is_available():
        # Set memory allocation strategy
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
        
        # Clear cache
        torch.cuda.empty_cache()
        
        # Get GPU info
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"🖥️  GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Total GPU Memory: {gpu_memory:.1f} GB")
        
        # Calculate optimal batch size for different GPU memory sizes
        if gpu_memory <= 4:
            recommended_batch = 4
            recommended_workers = 2
        elif gpu_memory <= 6:
            recommended_batch = 8
            recommended_workers = 4
        elif gpu_memory <= 8:
            recommended_batch = 16
            recommended_workers = 6
        else:
            recommended_batch = 32
            recommended_workers = 8
            
        return recommended_batch, recommended_workers
    else:
        return 8, 2  # CPU fallback

def train_ocr_optimized():
    print("🔤 Memory-Optimized YOLOv8 OCR Character Recognition Training")
    print("=" * 70)
    
    # Clear memory first
    clear_memory()
    
    # Optimize memory settings
    batch_size, workers = optimize_gpu_memory()
    
    # Dataset path - updated to match your structure
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Check if pretrained models exist in the directory
    pretrained_models = ['yolo11n.pt', 'yolov8n.pt']
    model_path = None
    
    for model_name in pretrained_models:
        model_file = os.path.join(dataset_path, model_name)
        if os.path.exists(model_file):
            model_path = model_file
            print(f"🤖 Found pretrained model: {model_name}")
            break
    
    if model_path is None:
        print("🤖 Using YOLOv8n (downloading if needed)...")
        model = YOLO('yolov8n.pt')
    else:
        print(f"🤖 Loading local model: {os.path.basename(model_path)}")
        model = YOLO(model_path)
    
    # OCR-optimized training configuration
    train_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 150,  # More epochs for character recognition
        'imgsz': 640,   # Good resolution for character details
        'batch': batch_size,
        'name': 'ocr_character_recognition',
        'patience': 50,  # More patience for character learning
        'save': True,
        'plots': True,
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': workers,
        'project': models_dir,
        'exist_ok': True,
        
        # Memory optimization
        'amp': True,
        'fraction': 0.9,
        'cache': False,  # Set to True if you have enough RAM
        'rect': True,
        'resume': False,
        'single_cls': False,
        
        # Learning rate parameters - optimized for character recognition
        'lr0': 0.01,    # Higher initial learning rate for character features
        'lrf': 0.01,    # Final learning rate
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        
        # Loss weights - optimized for character detection
        'box': 7.5,     # Box regression weight
        'cls': 0.5,     # Classification weight
        'dfl': 1.5,     # Distribution focal loss weight
        
        # Data augmentation - careful with characters
        'hsv_h': 0.015,  # Small hue variation
        'hsv_s': 0.7,    # Saturation variation
        'hsv_v': 0.4,    # Value variation
        'degrees': 2.0,  # Small rotation for characters
        'translate': 0.1, # Translation
        'scale': 0.5,    # Scale variation
        'shear': 2.0,    # Small shear for character variation
        'perspective': 0.0001, # Minimal perspective change
        'flipud': 0.0,   # No vertical flip for characters
        'fliplr': 0.0,   # No horizontal flip (would mirror characters)
        'mosaic': 1.0,   # Mosaic augmentation
        'mixup': 0.0,    # No mixup for character clarity
        'copy_paste': 0.1 # Small copy-paste augmentation
    }
    
    print("📋 OCR Character Recognition Configuration:")
    print(f"   Image size: {train_config['imgsz']}")
    print(f"   Batch size: {train_config['batch']}")
    print(f"   Workers: {train_config['workers']}")
    print(f"   Epochs: {train_config['epochs']}")
    print(f"   Classes: 36 (0-9, A-Z)")
    print(f"   AMP enabled: {train_config['amp']}")
    print(f"   Models will be saved to: {models_dir}")
    print("=" * 70)
    
    try:
        # Clear memory before training
        clear_memory()
        
        # Start training
        print("🚀 Starting OCR character recognition training...")
        results = model.train(**train_config)
        
        print("🎉 Training completed successfully!")
        
        # Get the actual save directory
        save_dir = os.path.join(models_dir, 'ocr_character_recognition')
        print(f"💾 Best model saved at: {save_dir}/weights/best.pt")
        
        # Clear memory after training
        clear_memory()
        
        # Quick validation
        print("\n🔍 Running validation...")
        metrics = model.val()
        
        print("📊 Validation Results:")
        if hasattr(metrics, 'box'):
            print(f"   mAP50: {metrics.box.map50:.4f}")
            print(f"   mAP50-95: {metrics.box.map:.4f}")
            print(f"   Precision: {metrics.box.mp:.4f}")
            print(f"   Recall: {metrics.box.mr:.4f}")
        else:
            print("   Validation metrics computed successfully")
        
        return save_dir
        
    except torch.cuda.OutOfMemoryError:
        print("❌ GPU out of memory! Trying CPU training...")
        clear_memory()
        return train_ocr_on_cpu()
    except Exception as e:
        print(f"❌ Training failed: {str(e)}")
        return None

def train_ocr_on_cpu():
    """Fallback to CPU training for OCR"""
    print("🖥️  Fallback: OCR Training on CPU...")
    
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Try to load local model first
    pretrained_models = ['yolo11n.pt', 'yolov8n.pt']
    model_path = None
    
    for model_name in pretrained_models:
        model_file = os.path.join(dataset_path, model_name)
        if os.path.exists(model_file):
            model_path = model_file
            break
    
    if model_path:
        model = YOLO(model_path)
    else:
        model = YOLO('yolov8n.pt')
    
    # CPU-optimized configuration for OCR
    cpu_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 100,
        'imgsz': 480,
        'batch': 8,
        'name': 'ocr_character_cpu',
        'patience': 30,
        'save': True,
        'plots': True,
        'device': 'cpu',
        'workers': 2,
        'project': models_dir,
        'exist_ok': True,
        
        # Minimal augmentation for CPU
        'hsv_h': 0.01,
        'hsv_s': 0.4,
        'hsv_v': 0.3,
        'degrees': 1.0,
        'translate': 0.05,
        'scale': 0.3,
        'fliplr': 0.0,  # No flip for characters
        'mosaic': 0.5,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    try:
        results = model.train(**cpu_config)
        print("🎉 CPU OCR training completed!")
        
        # Get save directory
        save_dir = os.path.join(models_dir, 'ocr_character_cpu')
        return save_dir
        
    except Exception as e:
        print(f"❌ CPU training also failed: {str(e)}")
        return None

def ultra_minimal_ocr_training():
    """Ultra minimal config for very limited resources"""
    print("🔬 Ultra Minimal OCR Training for Limited Resources")
    
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr"
    
    # Create models directory in dataset path
    models_dir = os.path.join(dataset_path, "models")
    os.makedirs(models_dir, exist_ok=True)
    
    # Clear all memory
    clear_memory()
    
    # Try to load local model first
    pretrained_models = ['yolo11n.pt', 'yolov8n.pt']
    model_path = None
    
    for model_name in pretrained_models:
        model_file = os.path.join(dataset_path, model_name)
        if os.path.exists(model_file):
            model_path = model_file
            break
    
    if model_path:
        model = YOLO(model_path)
    else:
        model = YOLO('yolov8n.pt')
    
    minimal_config = {
        'data': f'{dataset_path}/data.yaml',
        'epochs': 50,
        'imgsz': 320,
        'batch': 2,
        'name': 'ocr_character_minimal',
        'patience': 20,
        'save': True,
        'plots': False,
        'device': 0 if torch.cuda.is_available() else 'cpu',
        'workers': 0,
        'project': models_dir,
        'exist_ok': True,
        'cache': False,
        'amp': True,
        'fraction': 0.8,
        
        # Minimal augmentation
        'hsv_h': 0.0,
        'hsv_s': 0.0,
        'hsv_v': 0.0,
        'degrees': 0.0,
        'translate': 0.0,
        'scale': 0.0,
        'fliplr': 0.0,
        'mosaic': 0.0,
        'mixup': 0.0,
        'copy_paste': 0.0
    }
    
    print("⚡ Ultra minimal OCR configuration:")
    print(f"   Image size: {minimal_config['imgsz']}")
    print(f"   Batch size: {minimal_config['batch']}")
    print(f"   Epochs: {minimal_config['epochs']}")
    print(f"   Classes: 36 (0-9, A-Z)")
    print(f"   Models will be saved to: {models_dir}")
    
    try:
        results = model.train(**minimal_config)
        print("🎉 Ultra minimal OCR training completed!")
        
        # Get save directory
        save_dir = os.path.join(models_dir, 'ocr_character_minimal')
        return save_dir
        
    except Exception as e:
        print(f"❌ Even minimal training failed: {str(e)}")
        return None

def test_model_inference(model_path, test_image_path=None):
    """Test the trained model on sample images"""
    if not os.path.exists(model_path):
        print(f"❌ Model not found: {model_path}")
        return
    
    try:
        # Load the trained model
        model = YOLO(model_path)
        
        # Test directory
        dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr"
        test_dir = os.path.join(dataset_path, "images", "val")
        
        if test_image_path and os.path.exists(test_image_path):
            # Test specific image
            results = model(test_image_path)
            print(f"🔍 Testing on: {test_image_path}")
        elif os.path.exists(test_dir):
            # Test on validation images
            test_images = [f for f in os.listdir(test_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if test_images:
                test_image = os.path.join(test_dir, test_images[0])
                results = model(test_image)
                print(f"🔍 Testing on: {test_image}")
            else:
                print("❌ No test images found")
                return
        else:
            print("❌ No test directory found")
            return
        
        # Display results
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                print(f"📊 Detected {len(boxes)} characters:")
                for i, box in enumerate(boxes):
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    # Map class ID to character
                    chars = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
                            'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 
                            'U', 'V', 'W', 'X', 'Y', 'Z']
                    char = chars[class_id] if class_id < len(chars) else 'Unknown'
                    print(f"   {i+1}. Character: '{char}' (Confidence: {confidence:.2f})")
            else:
                print("📊 No characters detected")
        
    except Exception as e:
        print(f"❌ Inference failed: {str(e)}")

if __name__ == "__main__":
    # Set environment variable for memory optimization
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # Dataset path - updated for OCR
    dataset_path = "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr"
    
    print("🔤 OCR Character Recognition Training")
    print("Choose training mode:")
    print("1. Memory Optimized (Recommended)")
    print("2. CPU Training")
    print("3. Ultra Minimal")
    print("4. Test Trained Model")
    
    choice = input("Enter choice (1/2/3/4) [default: 1]: ").strip()
    
    if choice == "4":
        model_path = input("Enter path to trained model (best.pt): ").strip()
        if not model_path:
            # Try to find the latest trained model
            models_dir = os.path.join(dataset_path, "models")
            if os.path.exists(models_dir):
                for subdir in os.listdir(models_dir):
                    weights_dir = os.path.join(models_dir, subdir, "weights")
                    best_model = os.path.join(weights_dir, "best.pt")
                    if os.path.exists(best_model):
                        model_path = best_model
                        break
        
        if model_path and os.path.exists(model_path):
            test_model_inference(model_path)
        else:
            print("❌ No trained model found. Please train a model first.")
            
    elif choice == "2":
        save_dir = train_ocr_on_cpu()
    elif choice == "3":
        save_dir = ultra_minimal_ocr_training()
    else:
        save_dir = train_ocr_optimized()
    
    if choice in ["1", "2", "3"]:
        if save_dir:
            print(f"\n✅ OCR Training successful! Model saved at: {save_dir}")
            print(f"📁 Best weights: {save_dir}/weights/best.pt")
            print(f"📁 Last weights: {save_dir}/weights/last.pt")
            print(f"📁 Training results: {save_dir}")
            
            # Show directory structure
            print(f"\n📂 Directory structure:")
            print(f"   {dataset_path}/")
            print(f"   ├── models/")
            print(f"   │   └── {os.path.basename(save_dir)}/")
            print(f"   │       ├── weights/")
            print(f"   │       │   ├── best.pt")
            print(f"   │       │   └── last.pt")
            print(f"   │       ├── results.png")
            print(f"   │       └── confusion_matrix.png")
            print(f"   ├── images/")
            print(f"   │   ├── train/")
            print(f"   │   └── val/")
            print(f"   ├── labels/")
            print(f"   │   ├── train/")
            print(f"   │   └── val/")
            print(f"   └── data.yaml")
            
            # Offer to test the model
            test_choice = input("\n🔍 Would you like to test the trained model? (y/n): ").strip().lower()
            if test_choice == 'y':
                best_model = os.path.join(save_dir, "weights", "best.pt")
                test_model_inference(best_model)
            
        else:
            print("\n❌ All training attempts failed. Consider:")
            print("   - Checking your data.yaml file path")
            print("   - Ensuring images and labels are properly formatted")
            print("   - Using a machine with more GPU memory")
            print("   - Training on Google Colab with GPU")

🔤 OCR Character Recognition Training
Choose training mode:
1. Memory Optimized (Recommended)
2. CPU Training
3. Ultra Minimal
4. Test Trained Model


Enter choice (1/2/3/4) [default: 1]:  1


🔤 Memory-Optimized YOLOv8 OCR Character Recognition Training
🖥️  GPU: NVIDIA GeForce RTX 3050 Laptop GPU
💾 Total GPU Memory: 4.0 GB
🤖 Found pretrained model: yolo11n.pt
🤖 Loading local model: yolo11n.pt
📋 OCR Character Recognition Configuration:
   Image size: 640
   Batch size: 4
   Workers: 2
   Epochs: 150
   Classes: 36 (0-9, A-Z)
   AMP enabled: True
   Models will be saved to: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr\models
🚀 Starting OCR character recognition training...
New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.156  Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/minhk/Downloads/code thu/AI/Nguy

100%|█████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 13.3MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.20.1 ms, read: 3.51.1 MB/s, size: 24.0 KB)


...

                   all        767       6236      0.957      0.939      0.973      0.736
                     0        529        836       0.98      0.972      0.982       0.64
                     1        408        541       0.96      0.993      0.993      0.767
                     2        328        423       0.99      0.979      0.994      0.768
                     3        330        425      0.959      0.988      0.985      0.705
                     4        523        754      0.976      0.988       0.99      0.778
                     5        328        428      0.968      0.988      0.988      0.763
                     6        314        418      0.969       0.99      0.988       0.74
                     7        287        373      0.969      0.981      0.986      0.745
                     8        459        630      0.989      0.987      0.994      0.782
                     9         61         64      0.968      0.961      0.985      0.687
                     

val: Scanning C:\Users\minhk\Downloads\code thuê\AI\NguyenLam\datasetocr\labels\val.cache... 767 images, 2 backgrounds,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 192/192 [00:


                   all        767       6236      0.956      0.938      0.973      0.735
                     0        529        836       0.98      0.972      0.982      0.643
                     1        408        541      0.959      0.993      0.992      0.765
                     2        328        423       0.99      0.981      0.994       0.77
                     3        330        425      0.955      0.988      0.985      0.703
                     4        523        754      0.977      0.988      0.991      0.778
                     5        328        428       0.97      0.988      0.988      0.764
                     6        314        418      0.972       0.99      0.988      0.741
                     7        287        373      0.969      0.981      0.986      0.747
                     8        459        630       0.99      0.987      0.994      0.782
                     9         61         64      0.968      0.946      0.985      0.695
                     


🔍 Would you like to test the trained model? (y/n):  y



image 1/1 C:\Users\minhk\Downloads\code thu\AI\NguyenLam\datasetocr\images\val\10PlateBaza275.jpg: 384x640 1 0, 1 1, 1 2, 1 5, 1 6, 1 7, 1 9, 1 A, 1 F, 156.3ms
Speed: 28.0ms preprocess, 156.3ms inference, 30.2ms postprocess per image at shape (1, 3, 384, 640)
🔍 Testing on: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr\images\val\10PlateBaza275.jpg
📊 Detected 9 characters:
   1. Character: '2' (Confidence: 0.90)
   2. Character: 'A' (Confidence: 0.90)
   3. Character: '7' (Confidence: 0.89)
   4. Character: '1' (Confidence: 0.84)
   5. Character: 'F' (Confidence: 0.78)
   6. Character: '0' (Confidence: 0.69)
   7. Character: '9' (Confidence: 0.62)
   8. Character: '5' (Confidence: 0.55)
   9. Character: '6' (Confidence: 0.28)


In [5]:
import cv2
import torch
import numpy as np
import os
import time
from collections import deque
import warnings
import sys
import platform

# Suppress all warnings
warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['PYTHONWARNINGS'] = 'ignore'

# Suppress stdout/stderr for model downloads
from contextlib import redirect_stdout, redirect_stderr
import io

# Try to import both YOLOv5 and YOLOv8
try:
    from ultralytics import YOLO
    ULTRALYTICS_AVAILABLE = True
except ImportError:
    ULTRALYTICS_AVAILABLE = False
    print("⚠️ Ultralytics not available")

try:
    import yolov5
    YOLOV5_AVAILABLE = True
except ImportError:
    YOLOV5_AVAILABLE = False

# EasyOCR only
print("🔄 Checking EasyOCR...")
try:
    import easyocr
    EASYOCR_AVAILABLE = True
    print("✅ EasyOCR available")
except ImportError:
    EASYOCR_AVAILABLE = False
    print("❌ EasyOCR not available")
    sys.exit("EasyOCR is required! Install with: pip install easyocr")

class EasyOCREngine:
    """Simplified OCR Engine using only EasyOCR"""
    
    def __init__(self):
        self.ocr_reader = None
        self.total_uses = 0
        self.successful_detections = 0
        self.total_confidence = 0.0
        self.total_time = 0.0
        self.initialize_ocr()
    
    def initialize_ocr(self):
        """Initialize EasyOCR"""
        print("🔄 Initializing EasyOCR...")
        try:
            with redirect_stdout(io.StringIO()), redirect_stderr(io.StringIO()):
                self.ocr_reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
            print("✅ EasyOCR initialized successfully")
        except Exception as e:
            print(f"❌ Failed to initialize EasyOCR: {e}")
            sys.exit("Failed to initialize EasyOCR")
    
    def preprocess_for_ocr(self, image):
        """Preprocess image for better OCR results"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Try different preprocessing methods
        processed_images = [('original', gray)]
        
        # Gaussian blur + OTSU threshold
        try:
            blurred = cv2.GaussianBlur(gray, (3, 3), 0)
            _, thresh1 = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            processed_images.append(('otsu', thresh1))
        except:
            pass
        
        # Adaptive threshold
        try:
            adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
            processed_images.append(('adaptive', adaptive))
        except:
            pass
        
        # Histogram equalization
        try:
            equalized = cv2.equalizeHist(gray)
            processed_images.append(('equalized', equalized))
        except:
            pass
        
        return processed_images
    
    def recognize_with_easyocr(self, image):
        """OCR using EasyOCR"""
        try:
            with redirect_stdout(io.StringIO()), redirect_stderr(io.StringIO()):
                results = self.ocr_reader.readtext(image)
            
            text_parts = []
            confidences = []
            
            for (bbox, text, confidence) in results:
                if confidence > 0.3:  # Filter low confidence results
                    clean_text = ''.join(c for c in text.upper() if c.isalnum())
                    if clean_text:
                        text_parts.append(clean_text)
                        confidences.append(confidence)
            
            final_text = ''.join(text_parts)
            avg_confidence = np.mean(confidences) if confidences else 0
            
            return final_text, avg_confidence
        
        except Exception:
            return "", 0
    
    def recognize_text(self, image):
        """Recognize text using EasyOCR with multiple preprocessing methods"""
        start_time = time.time()
        processed_images = self.preprocess_for_ocr(image)
        best_result = ("", 0)
        
        # Try different preprocessing methods
        for preprocess_name, processed_img in processed_images:
            try:
                text, conf = self.recognize_with_easyocr(processed_img)
                
                if conf > best_result[1] and len(text) >= 3:
                    best_result = (text, conf)
                    
            except Exception:
                continue
        
        # Update statistics
        processing_time = time.time() - start_time
        self.total_uses += 1
        self.total_time += processing_time
        
        if len(best_result[0]) >= 3 and best_result[1] > 0:
            self.successful_detections += 1
            self.total_confidence += best_result[1]
        
        return best_result
    
    def get_stats(self):
        """Get OCR statistics"""
        avg_time = self.total_time / max(self.total_uses, 1)
        success_rate = (self.successful_detections / max(self.total_uses, 1)) * 100
        avg_confidence = self.total_confidence / max(self.successful_detections, 1)
        
        return {
            'total_uses': self.total_uses,
            'successful_detections': self.successful_detections,
            'success_rate': success_rate,
            'avg_confidence': avg_confidence,
            'avg_time': avg_time
        }
    
    def reset_stats(self):
        """Reset statistics"""
        self.total_uses = 0
        self.successful_detections = 0
        self.total_confidence = 0.0
        self.total_time = 0.0
        print("🗑️ OCR statistics reset")

class LicensePlateDetector:
    def __init__(self, plate_model_path):
        self.plate_model_path = plate_model_path
        
        print("🔄 Loading models...")
        
        # Load YOLO model
        try:
            self.plate_model, self.plate_model_type = self.load_model(plate_model_path)
            if self.plate_model:
                print(f"✅ License plate model loaded: {os.path.basename(plate_model_path)} ({self.plate_model_type})")
            else:
                print(f"❌ Failed to load plate model: {plate_model_path}")
        except Exception as e:
            print(f"❌ Failed to load plate model: {e}")
            self.plate_model = None
            self.plate_model_type = None
        
        # Initialize EasyOCR
        self.ocr_engine = EasyOCREngine()
        
        self.plate_conf_threshold = 0.3
        self.recent_plates = deque(maxlen=10)
        
        # FPS tracking
        self.fps_counter = 0
        self.fps_start_time = time.time()
        self.current_fps = 0
    
    def load_model(self, model_path):
        """Load YOLO model with automatic detection"""
        if not os.path.exists(model_path):
            return None, None
        
        # Try YOLOv8 first
        if ULTRALYTICS_AVAILABLE:
            try:
                model = YOLO(model_path)
                return model, "YOLOv8"
            except Exception:
                pass
        
        # Try YOLOv5
        if YOLOV5_AVAILABLE:
            try:
                model = yolov5.load(model_path)
                return model, "YOLOv5"
            except Exception:
                pass
        
        # Try YOLOv5 from torch hub
        try:
            model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
            return model, "YOLOv5-torch"
        except Exception:
            pass
        
        return None, None
    
    def run_inference(self, model, model_type, image, conf_threshold):
        """Run inference based on model type"""
        try:
            if model_type == "YOLOv5" or model_type == "YOLOv5-torch":
                results = model(image, size=640)
                return self.parse_yolov5_results(results, conf_threshold)
            elif model_type == "YOLOv8":
                results = model(image, conf=conf_threshold, verbose=False)
                return self.parse_yolov8_results(results)
            else:
                return []
        except Exception:
            return []
    
    def parse_yolov5_results(self, results, conf_threshold):
        """Parse YOLOv5 results"""
        detections = []
        df = results.pandas().xyxy[0]
        
        for index, row in df.iterrows():
            confidence = row['confidence']
            if confidence >= conf_threshold:
                x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                class_id = int(row['class'])
                
                detections.append({
                    'bbox': (x1, y1, x2, y2),
                    'confidence': confidence,
                    'class_id': class_id
                })
        
        return detections
    
    def parse_yolov8_results(self, results):
        """Parse YOLOv8 results"""
        detections = []
        
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    
                    detections.append({
                        'bbox': (x1, y1, x2, y2),
                        'confidence': confidence,
                        'class_id': class_id
                    })
        
        return detections
    
    def detect_license_plates(self, frame):
        """Detect license plates in frame"""
        if self.plate_model is None:
            return []
        
        try:
            detections = self.run_inference(
                self.plate_model, 
                self.plate_model_type, 
                frame, 
                self.plate_conf_threshold
            )
            
            plates = []
            for detection in detections:
                x1, y1, x2, y2 = detection['bbox']
                confidence = detection['confidence']
                
                # Add padding to the bounding box
                h, w = frame.shape[:2]
                pad = 5
                x1 = max(0, x1 - pad)
                y1 = max(0, y1 - pad)
                x2 = min(w, x2 + pad)
                y2 = min(h, y2 + pad)
                
                plate_roi = frame[y1:y2, x1:x2]
                
                plates.append({
                    'bbox': (x1, y1, x2, y2),
                    'confidence': confidence,
                    'roi': plate_roi
                })
            
            return plates
        except Exception:
            return []
    
    def recognize_text(self, plate_roi):
        """Recognize text in license plate ROI"""
        if plate_roi is None or plate_roi.size == 0:
            return "", 0
        
        # Resize small images for better OCR
        h, w = plate_roi.shape[:2]
        if h < 32 or w < 100:
            scale = max(32/h, 100/w)
            plate_roi = cv2.resize(plate_roi, (int(w*scale), int(h*scale)))
        
        return self.ocr_engine.recognize_text(plate_roi)
    
    def draw_results(self, frame, plates_info):
        """Draw detection results on frame"""
        for plate_info in plates_info:
            x1, y1, x2, y2 = plate_info['bbox']
            confidence = plate_info['confidence']
            plate_text = plate_info.get('text', '')
            ocr_confidence = plate_info.get('ocr_confidence', 0)
            
            # Color: green if text detected, yellow if only plate detected
            color = (0, 255, 0) if plate_text else (0, 255, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw plate confidence
            cv2.putText(frame, f'Plate: {confidence:.2f}', 
                       (x1, y1-30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Draw recognized text
            if plate_text:
                text_size = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
                cv2.rectangle(frame, (x1, y1-60), (x1 + text_size[0] + 10, y1-35), (0, 0, 0), -1)
                
                cv2.putText(frame, plate_text, 
                           (x1+5, y1-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                cv2.putText(frame, f'OCR: {ocr_confidence:.2f}', 
                           (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def update_fps(self):
        """Update FPS counter"""
        self.fps_counter += 1
        current_time = time.time()
        elapsed = current_time - self.fps_start_time
        
        if elapsed >= 1.0:
            self.current_fps = self.fps_counter / elapsed
            self.fps_counter = 0
            self.fps_start_time = current_time
    
    def process_frame(self, frame):
        """Process single frame for license plate detection and OCR"""
        plates = self.detect_license_plates(frame)
        
        plates_info = []
        for plate in plates:
            plate_info = {
                'bbox': plate['bbox'],
                'confidence': plate['confidence']
            }
            
            # Perform OCR if plate ROI is valid
            if plate['roi'] is not None and plate['roi'].size > 0:
                text, ocr_conf = self.recognize_text(plate['roi'])
                if text:
                    plate_info['text'] = text
                    plate_info['ocr_confidence'] = ocr_conf
                    
                    # Store successful detection
                    if len(text) >= 3:
                        detection = {
                            'text': text,
                            'confidence': plate['confidence'],
                            'ocr_confidence': ocr_conf,
                            'timestamp': time.time()
                        }
                        self.recent_plates.append(detection)
            
            plates_info.append(plate_info)
        
        self.update_fps()
        return plates_info
    
    def get_ocr_stats(self):
        """Get OCR statistics"""
        return self.ocr_engine.get_stats()
    
    def reset_ocr_stats(self):
        """Reset OCR statistics"""
        self.ocr_engine.reset_stats()

def find_trained_models(base_path):
    """Find trained models in the dataset directory"""
    models = {'plate_model': None}
    
    search_paths = [
        os.path.join(base_path, "models"),
        os.path.join(base_path, "dataset", "models"),
        os.path.join(base_path, "datasetocr", "models"),
        base_path
    ]
    
    for search_path in search_paths:
        if not os.path.exists(search_path):
            continue
            
        if os.path.isdir(search_path):
            for file in os.listdir(search_path):
                if file.endswith('.pt'):
                    full_path = os.path.join(search_path, file)
                    if 'plate' in file.lower() or 'red' in file.lower():
                        models['plate_model'] = full_path
        
        # Look for weights/best.pt in subdirectories
        try:
            for subdir in os.listdir(search_path):
                subdir_path = os.path.join(search_path, subdir)
                if os.path.isdir(subdir_path):
                    weights_dir = os.path.join(subdir_path, "weights")
                    if os.path.exists(weights_dir):
                        best_model = os.path.join(weights_dir, "best.pt")
                        if os.path.exists(best_model):
                            if 'plate' in subdir.lower() or 'red' in subdir.lower():
                                models['plate_model'] = best_model
        except PermissionError:
            continue
    
    return models

def main():
    print("\n🚗 License Plate Detection & OCR System (EasyOCR Only)")
    print("=" * 60)
    
    if not EASYOCR_AVAILABLE:
        print("❌ EasyOCR is not available!")
        print("Install with: pip install easyocr")
        return
    
    # Find model paths
    base_paths = [
        "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam",
        "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset",
        "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr"
    ]
    
    models = {'plate_model': None}
    
    for base_path in base_paths:
        if os.path.exists(base_path):
            found_models = find_trained_models(base_path)
            if found_models['plate_model'] and not models['plate_model']:
                models['plate_model'] = found_models['plate_model']
    
    print(f"\n🔍 Found models:")
    print(f"   License Plate Model: {models['plate_model'] or 'Not found'}")
    
    # Ask for model path if not found
    if not models['plate_model']:
        plate_path = input("Enter path to license plate detection model (.pt file): ").strip()
        if os.path.exists(plate_path):
            models['plate_model'] = plate_path
        else:
            print("❌ License plate model not found!")
            return
    
    # Initialize detector
    detector = LicensePlateDetector(models['plate_model'])
    
    # Initialize webcam
    print("\n📷 Initializing webcam...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Cannot open webcam!")
        return
    
    # Set webcam properties
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("✅ Webcam initialized successfully!")
    print(f"\n🎯 Controls:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save current frame")
    print("   - Press 'r' to show recent detections")
    print("   - Press 'c' to clear detection history")
    print("   - Press 'i' to show OCR statistics")
    print("   - Press 'x' to reset OCR statistics")
    
    frame_count = 0
    save_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to read frame from webcam")
                break
            
            frame_count += 1
            
            # Process frame
            plates_info = detector.process_frame(frame)
            frame = detector.draw_results(frame, plates_info)
            
            # Draw info overlay
            info_y = 30
            cv2.putText(frame, f'FPS: {detector.current_fps:.1f}', 
                       (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.putText(frame, f'Plates: {len(plates_info)}', 
                       (10, info_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.putText(frame, f'Recent: {len(detector.recent_plates)}', 
                       (10, info_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.putText(frame, 'OCR: EasyOCR', 
                       (10, info_y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            # Show OCR stats
            stats = detector.get_ocr_stats()
            cv2.putText(frame, f'OCR Stats: {stats["total_uses"]} uses, {stats["success_rate"]:.1f}% success', 
                       (10, info_y + 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
            
            # Show latest detection
            if detector.recent_plates:
                latest = detector.recent_plates[-1]
                cv2.putText(frame, f'Latest: {latest["text"]} ({latest["ocr_confidence"]:.2f})', 
                           (10, info_y + 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            
            cv2.imshow('License Plate Detection & OCR - EasyOCR', frame)
            
            # Handle key press
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('s'):
                save_count += 1
                filename = f"detection_frame_{save_count:04d}.jpg"
                cv2.imwrite(filename, frame)
                print(f"💾 Frame saved: {filename}")
            elif key == ord('r'):
                print("\n📋 Recent License Plate Detections:")
                if detector.recent_plates:
                    for i, detection in enumerate(reversed(list(detector.recent_plates))):
                        timestamp = time.strftime('%H:%M:%S', time.localtime(detection['timestamp']))
                        print(f"   {i+1:2d}. {detection['text']:15s} | "
                              f"Plate: {detection['confidence']:.2f} | "
                              f"OCR: {detection['ocr_confidence']:.2f} | "
                              f"{timestamp}")
                else:
                    print("   No detections yet")
            elif key == ord('c'):
                detector.recent_plates.clear()
                print("🗑️ Detection history cleared")
            elif key == ord('i'):
                stats = detector.get_ocr_stats()
                print(f"\n📊 EasyOCR Statistics:")
                print(f"   Total uses: {stats['total_uses']}")
                print(f"   Successful detections: {stats['successful_detections']}")
                print(f"   Success rate: {stats['success_rate']:.1f}%")
                print(f"   Average confidence: {stats['avg_confidence']:.3f}")
                print(f"   Average processing time: {stats['avg_time']:.3f}s")
            elif key == ord('x'):
                detector.reset_ocr_stats()
    
    except KeyboardInterrupt:
        print("\n⏹️ Interrupted by user")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Webcam released and windows closed")
        
        # Final statistics
        stats = detector.get_ocr_stats()
        print(f"\n📊 Final Session Statistics:")
        print(f"   Total frames processed: {frame_count}")
        print(f"   Total detections: {len(detector.recent_plates)}")
        print(f"   Frames saved: {save_count}")
        print(f"   EasyOCR uses: {stats['total_uses']}")
        print(f"   EasyOCR success rate: {stats['success_rate']:.1f}%")
        print(f"   EasyOCR average confidence: {stats['avg_confidence']:.3f}")

if __name__ == "__main__":
    main()

🔄 Checking EasyOCR...
✅ EasyOCR available

🚗 License Plate Detection & OCR System (EasyOCR Only)

🔍 Found models:
   License Plate Model: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt
🔄 Loading models...
✅ License plate model loaded: best.pt (YOLOv8)
🔄 Initializing EasyOCR...
✅ EasyOCR initialized successfully

📷 Initializing webcam...
✅ Webcam initialized successfully!

🎯 Controls:
   - Press 'q' to quit
   - Press 's' to save current frame
   - Press 'r' to show recent detections
   - Press 'c' to clear detection history
   - Press 'i' to show OCR statistics
   - Press 'x' to reset OCR statistics
✅ Webcam released and windows closed

📊 Final Session Statistics:
   Total frames processed: 45742
   Total detections: 10
   Frames saved: 0
   EasyOCR uses: 1530
   EasyOCR success rate: 71.2%
   EasyOCR average confidence: 0.735


In [1]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import os
import time
from collections import deque
import threading
import queue
from pathlib import Path
import sys

# Global variable for YOLOv5 availability
YOLOV5_AVAILABLE = False

def check_and_install_yolov5():
    """Check and install YOLOv5 if needed"""
    global YOLOV5_AVAILABLE
    try:
        # Try different ways to import YOLOv5
        try:
            import yolov5
            YOLOV5_AVAILABLE = True
            print("✅ YOLOv5 package available")
            return True
        except ImportError:
            pass
        
        # Try to install yolov5
        print("📦 Installing YOLOv5 package...")
        import subprocess
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'yolov5'])
        
        try:
            import yolov5
            YOLOV5_AVAILABLE = True
            print("✅ YOLOv5 package installed and available")
            return True
        except ImportError:
            print("❌ Failed to import yolov5 after installation")
            return False
            
    except Exception as e:
        print(f"❌ Failed to install YOLOv5: {e}")
        return False

# Check YOLOv5 availability at startup
check_and_install_yolov5()

class YOLOv5Wrapper:
    """Wrapper class for YOLOv5 model to handle Windows path issues"""
    
    def __init__(self, model_path):
        self.model_path = str(model_path).replace('\\', '/')
        self.model = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Try multiple ways to load YOLOv5 model
        self._load_model()
    
    def _load_model(self):
        """Try different methods to load YOLOv5 model"""
        
        # Method 1: Direct torch.load
        try:
            print(f"🔄 Trying torch.load for: {self.model_path}")
            checkpoint = torch.load(self.model_path, map_location=self.device)
            
            # If it's a YOLOv5 model dict
            if isinstance(checkpoint, dict) and 'model' in checkpoint:
                # Try to reconstruct model from checkpoint
                if YOLOV5_AVAILABLE:
                    import yolov5
                    self.model = yolov5.load(self.model_path, device=self.device)
                    print("✅ YOLOv5 model loaded via yolov5 package")
                    return
            else:
                print("⚠️ Model format not recognized for torch.load")
                
        except Exception as e:
            print(f"❌ torch.load failed: {e}")
        
        # Method 2: YOLOv5 package
        if YOLOV5_AVAILABLE:
            try:
                import yolov5
                self.model = yolov5.load(self.model_path, device=self.device)
                print("✅ YOLOv5 model loaded via yolov5 package")
                return
            except Exception as e:
                print(f"❌ yolov5.load failed: {e}")
        
        # Method 3: Try to load as Ultralytics format anyway
        try:
            from ultralytics import YOLO
            self.model = YOLO(self.model_path)
            print("✅ Model loaded as Ultralytics YOLO (might work)")
            return
        except Exception as e:
            print(f"❌ Ultralytics YOLO failed: {e}")
        
        # Method 4: Manual model reconstruction
        try:
            self._manual_load()
        except Exception as e:
            print(f"❌ Manual load failed: {e}")
            
        if self.model is None:
            raise Exception("Failed to load YOLOv5 model with all methods")
    
    def _manual_load(self):
        """Manual model loading for YOLOv5"""
        print("🔄 Trying manual YOLOv5 model loading...")
        
        # Load checkpoint
        checkpoint = torch.load(self.model_path, map_location=self.device)
        
        # Try to create a basic wrapper
        class YOLOv5Model:
            def __init__(self, checkpoint, device):
                self.checkpoint = checkpoint
                self.device = device
                self.model = checkpoint.get('model', checkpoint)
                if hasattr(self.model, 'to'):
                    self.model = self.model.to(device)
                    self.model.eval()
            
            def __call__(self, img):
                # Basic inference wrapper
                if hasattr(self.model, 'forward'):
                    with torch.no_grad():
                        results = self.model(img)
                    return self._format_results(results)
                else:
                    raise Exception("Model doesn't have forward method")
            
            def _format_results(self, results):
                # Format results to match expected output
                class Results:
                    def __init__(self, results):
                        self.xyxy = [results] if not isinstance(results, list) else results
                
                return Results(results)
        
        self.model = YOLOv5Model(checkpoint, self.device)
        print("✅ Manual YOLOv5 model loading successful")
    
    def __call__(self, img):
        """Run inference"""
        if self.model is None:
            raise Exception("Model not loaded")
        
        return self.model(img)

class LicensePlateDetector:
    def __init__(self, plate_model_path, ocr_model_path):
        """
        Initialize License Plate Detection and OCR system
        
        Args:
            plate_model_path: Path to trained license plate detection model
            ocr_model_path: Path to trained OCR character recognition model
        """
        self.plate_model_path = plate_model_path
        self.ocr_model_path = ocr_model_path
        
        # Load models
        print("🔄 Loading models...")
        
        # Load plate detection model (usually YOLOv8)
        try:
            self.plate_model = YOLO(plate_model_path)
            print(f"✅ License plate model loaded: {os.path.basename(plate_model_path)}")
        except Exception as e:
            print(f"❌ Failed to load plate model: {e}")
            self.plate_model = None
        
        # Load OCR model with improved error handling
        self.ocr_model = None
        self.ocr_model_type = None
        
        self._load_ocr_model()
        
        # Character mapping for OCR
        self.char_map = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
                        'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 
                        'U', 'V', 'W', 'X', 'Y', 'Z']
        
        # Detection parameters
        self.plate_conf_threshold = 0.3
        self.ocr_conf_threshold = 0.2
        
        # Results storage
        self.recent_plates = deque(maxlen=10)
        self.detection_history = []
        
        # Performance tracking
        self.fps_counter = 0
        self.fps_start_time = time.time()
        self.current_fps = 0
    
    def _load_ocr_model(self):
        """Load OCR model with multiple fallback methods"""
        
        # Convert path to handle Windows paths properly
        ocr_path = str(self.ocr_model_path).replace('\\', '/')
        
        print(f"🔄 Loading OCR model from: {ocr_path}")
        
        # Method 1: Try as YOLOv8
        try:
            self.ocr_model = YOLO(ocr_path)
            self.ocr_model_type = "yolov8"
            print(f"✅ OCR model loaded as YOLOv8: {os.path.basename(ocr_path)}")
            return
        except Exception as e:
            print(f"⚠️ YOLOv8 loading failed: {e}")
        
        # Method 2: Try YOLOv5 wrapper
        try:
            self.ocr_model = YOLOv5Wrapper(ocr_path)
            self.ocr_model_type = "yolov5_wrapper"
            print(f"✅ OCR model loaded as YOLOv5 wrapper: {os.path.basename(ocr_path)}")
            return
        except Exception as e:
            print(f"⚠️ YOLOv5 wrapper loading failed: {e}")
        
        # Method 3: Direct YOLOv5 if available
        if YOLOV5_AVAILABLE:
            try:
                import yolov5
                self.ocr_model = yolov5.load(ocr_path)
                self.ocr_model_type = "yolov5"
                print(f"✅ OCR model loaded as YOLOv5: {os.path.basename(ocr_path)}")
                return
            except Exception as e:
                print(f"⚠️ Direct YOLOv5 loading failed: {e}")
        
        # Method 4: Try loading with torch directly
        try:
            checkpoint = torch.load(ocr_path, map_location='cpu')
            print("✅ Model checkpoint loaded successfully")
            print("⚠️ Manual inference implementation needed")
            # You would need to implement manual inference here
            # For now, we'll skip this
        except Exception as e:
            print(f"❌ Direct torch loading failed: {e}")
        
        print("❌ All OCR model loading methods failed")
    
    def detect_license_plates(self, frame):
        """Detect license plates in frame"""
        if self.plate_model is None:
            return []
        
        try:
            results = self.plate_model(frame, conf=self.plate_conf_threshold, verbose=False)
            plates = []
            
            for result in results:
                if result.boxes is not None:
                    for box in result.boxes:
                        # Get bounding box coordinates
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        confidence = float(box.conf[0])
                        
                        # Extract plate region
                        plate_roi = frame[y1:y2, x1:x2]
                        
                        plates.append({
                            'bbox': (x1, y1, x2, y2),
                            'confidence': confidence,
                            'roi': plate_roi
                        })
            
            return plates
        except Exception as e:
            print(f"❌ Plate detection error: {e}")
            return []
    
    def recognize_characters(self, plate_roi):
        """Recognize characters in license plate ROI"""
        if self.ocr_model is None or plate_roi is None or plate_roi.size == 0:
            return "", 0, []
        
        try:
            # Preprocess plate ROI
            if len(plate_roi.shape) == 3:
                plate_gray = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2GRAY)
            else:
                plate_gray = plate_roi
            
            # Enhance contrast
            plate_enhanced = cv2.equalizeHist(plate_gray)
            
            # Convert back to BGR for YOLO
            plate_bgr = cv2.cvtColor(plate_enhanced, cv2.COLOR_GRAY2BGR)
            
            # Resize if too small
            h, w = plate_bgr.shape[:2]
            if h < 32 or w < 100:
                scale = max(32/h, 100/w)
                plate_bgr = cv2.resize(plate_bgr, (int(w*scale), int(h*scale)))
            
            characters = []
            
            # Handle different model types
            if self.ocr_model_type == "yolov8":
                characters = self._recognize_yolov8(plate_bgr)
            elif self.ocr_model_type == "yolov5":
                characters = self._recognize_yolov5(plate_bgr)
            elif self.ocr_model_type == "yolov5_wrapper":
                characters = self._recognize_yolov5_wrapper(plate_bgr)
            else:
                print(f"⚠️ Unknown model type: {self.ocr_model_type}")
                return "", 0, []
            
            # Sort and build text
            characters.sort(key=lambda x: x['center_x'])
            plate_text = ''.join([char['char'] for char in characters])
            avg_confidence = np.mean([char['confidence'] for char in characters]) if characters else 0
            
            return plate_text, avg_confidence, characters
            
        except Exception as e:
            print(f"❌ OCR error: {e}")
            return "", 0, []
    
    def _recognize_yolov8(self, plate_bgr):
        """YOLOv8 recognition"""
        results = self.ocr_model(plate_bgr, conf=self.ocr_conf_threshold, verbose=False)
        characters = []
        
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    if class_id < len(self.char_map):
                        char = self.char_map[class_id]
                        characters.append({
                            'char': char,
                            'confidence': confidence,
                            'bbox': (x1, y1, x2, y2),
                            'center_x': (x1 + x2) / 2
                        })
        
        return characters
    
    def _recognize_yolov5(self, plate_bgr):
        """YOLOv5 recognition"""
        results = self.ocr_model(plate_bgr)
        characters = []
        
        # Handle different YOLOv5 result formats
        try:
            if hasattr(results, 'pandas'):
                df = results.pandas().xyxy[0]
                for _, row in df.iterrows():
                    if row['confidence'] >= self.ocr_conf_threshold:
                        class_id = int(row['class'])
                        confidence = float(row['confidence'])
                        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                        
                        if class_id < len(self.char_map):
                            char = self.char_map[class_id]
                            characters.append({
                                'char': char,
                                'confidence': confidence,
                                'bbox': (x1, y1, x2, y2),
                                'center_x': (x1 + x2) / 2
                            })
            else:
                # Tensor format
                for *box, conf, cls in results.xyxy[0].cpu().numpy():
                    if conf >= self.ocr_conf_threshold:
                        class_id = int(cls)
                        confidence = float(conf)
                        x1, y1, x2, y2 = map(int, box)
                        
                        if class_id < len(self.char_map):
                            char = self.char_map[class_id]
                            characters.append({
                                'char': char,
                                'confidence': confidence,
                                'bbox': (x1, y1, x2, y2),
                                'center_x': (x1 + x2) / 2
                            })
        except Exception as e:
            print(f"⚠️ YOLOv5 result parsing error: {e}")
        
        return characters
    
    def _recognize_yolov5_wrapper(self, plate_bgr):
        """YOLOv5 wrapper recognition"""
        try:
            results = self.ocr_model(plate_bgr)
            # This would need custom implementation based on the wrapper
            # For now, return empty
            return []
        except Exception as e:
            print(f"⚠️ YOLOv5 wrapper error: {e}")
            return []
    
    def draw_results(self, frame, plates_info):
        """Draw detection results on frame"""
        for plate_info in plates_info:
            x1, y1, x2, y2 = plate_info['bbox']
            confidence = plate_info['confidence']
            plate_text = plate_info.get('text', '')
            ocr_confidence = plate_info.get('ocr_confidence', 0)
            
            # Draw license plate bounding box
            color = (0, 255, 0) if plate_text else (0, 255, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw plate detection confidence
            cv2.putText(frame, f'Plate: {confidence:.2f}', 
                       (x1, y1-30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Draw recognized text
            if plate_text:
                text_size = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
                cv2.rectangle(frame, (x1, y1-60), (x1 + text_size[0] + 10, y1-35), (0, 0, 0), -1)
                
                cv2.putText(frame, plate_text, 
                           (x1+5, y1-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                cv2.putText(frame, f'OCR: {ocr_confidence:.2f}', 
                           (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Draw model info
        model_info = f"Plate: YOLOv8 | OCR: {self.ocr_model_type or 'None'}"
        cv2.putText(frame, model_info, (10, frame.shape[0] - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def update_fps(self):
        """Update FPS counter"""
        self.fps_counter += 1
        current_time = time.time()
        elapsed = current_time - self.fps_start_time
        
        if elapsed >= 1.0:
            self.current_fps = self.fps_counter / elapsed
            self.fps_counter = 0
            self.fps_start_time = current_time
    
    def process_frame(self, frame):
        """Process single frame for license plate detection and OCR"""
        plates = self.detect_license_plates(frame)
        
        plates_info = []
        for plate in plates:
            plate_info = {
                'bbox': plate['bbox'],
                'confidence': plate['confidence']
            }
            
            if plate['roi'] is not None and plate['roi'].size > 0:
                ocr_result = self.recognize_characters(plate['roi'])
                if len(ocr_result) == 3:
                    text, ocr_conf, characters = ocr_result
                    plate_info['text'] = text
                    plate_info['ocr_confidence'] = ocr_conf
                    plate_info['characters'] = characters
                    
                    if text and len(text) >= 3:
                        detection = {
                            'text': text,
                            'confidence': plate['confidence'],
                            'ocr_confidence': ocr_conf,
                            'timestamp': time.time()
                        }
                        self.recent_plates.append(detection)
            
            plates_info.append(plate_info)
        
        self.update_fps()
        return plates_info

def find_trained_models(base_path):
    """Find trained models in the dataset directory"""
    models = {
        'plate_model': None,
        'ocr_model': None
    }
    
    # Specific search for OCR character detection model
    ocr_model_paths = [
        os.path.join(base_path, "datasetocr", "models", "ocr_character_recognition", "weights", "ocr_character_detection_best.pt"),
        os.path.join(base_path, "models", "ocr_character_recognition", "weights", "ocr_character_detection_best.pt"),
        os.path.join(base_path, "ocr_character_recognition", "weights", "ocr_character_detection_best.pt"),
        os.path.join(base_path, "datasetocr", "models", "ocr_character_recognition", "weights", "best.pt"),
        os.path.join(base_path, "models", "ocr_character_recognition", "weights", "best.pt"),
    ]
    
    for ocr_path in ocr_model_paths:
        if os.path.exists(ocr_path):
            models['ocr_model'] = ocr_path
            print(f"🎯 Found OCR model: {ocr_path}")
            break
    
    # Look for license plate model
    search_paths = [
        os.path.join(base_path, "models"),
        os.path.join(base_path, "dataset", "models"),
        os.path.join(base_path, "datasetocr", "models"),
        base_path
    ]
    
    for search_path in search_paths:
        if not os.path.exists(search_path):
            continue
            
        for subdir in os.listdir(search_path):
            subdir_path = os.path.join(search_path, subdir)
            if os.path.isdir(subdir_path):
                weights_dir = os.path.join(subdir_path, "weights")
                if os.path.exists(weights_dir):
                    best_model = os.path.join(weights_dir, "best.pt")
                    if os.path.exists(best_model):
                        if 'ocr' not in subdir.lower() and 'character' not in subdir.lower():
                            if 'plate' in subdir.lower() or 'red' in subdir.lower() or not models['plate_model']:
                                models['plate_model'] = best_model
                                print(f"🎯 Found plate model: {best_model}")
    
    return models

def main():
    print("🚗 License Plate Detection and OCR System")
    print("=" * 50)
    
    # Base paths
    base_paths = [
        "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam",
        "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset",
        "C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr",
    ]
    
    # Find trained models
    models = {'plate_model': None, 'ocr_model': None}
    
    for base_path in base_paths:
        if os.path.exists(base_path):
            print(f"🔍 Searching in: {base_path}")
            found_models = find_trained_models(base_path)
            if found_models['plate_model'] and not models['plate_model']:
                models['plate_model'] = found_models['plate_model']
            if found_models['ocr_model'] and not models['ocr_model']:
                models['ocr_model'] = found_models['ocr_model']
    
    print("\n📁 Model Status:")
    print(f"   License Plate Model: {models['plate_model'] or '❌ Not found'}")
    print(f"   OCR Model: {models['ocr_model'] or '❌ Not found'}")
    
    if not models['plate_model'] or not models['ocr_model']:
        print("❌ Required models not found!")
        return
    
    print(f"\n✅ Using models:")
    print(f"   Plate: {os.path.basename(models['plate_model'])}")
    print(f"   OCR: {os.path.basename(models['ocr_model'])}")
    
    # Initialize detector
    detector = LicensePlateDetector(models['plate_model'], models['ocr_model'])
    
    # Initialize webcam
    print("\n📷 Initializing webcam...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Cannot open webcam!")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("✅ Webcam initialized successfully!")
    print(f"🔧 Using OCR model type: {detector.ocr_model_type}")
    print("\n🎯 Controls:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save current frame")
    print("   - Press 'r' to show recent detections")
    print("   - Press 'c' to clear detection history")
    
    frame_count = 0
    save_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to read frame from webcam")
                break
            
            frame_count += 1
            
            # Process frame (only plate detection if OCR failed)
            if detector.ocr_model is not None:
                plates_info = detector.process_frame(frame)
            else:
                # Only do plate detection
                plates = detector.detect_license_plates(frame)
                plates_info = [{'bbox': p['bbox'], 'confidence': p['confidence']} for p in plates]
            
            # Draw results
            frame = detector.draw_results(frame, plates_info)
            
            # Draw info panel
            info_y = 30
            cv2.putText(frame, f'FPS: {detector.current_fps:.1f}', 
                       (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.putText(frame, f'Plates Detected: {len(plates_info)}', 
                       (10, info_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            if detector.ocr_model is None:
                cv2.putText(frame, 'OCR: Not Available', 
                           (10, info_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow('License Plate Detection & OCR', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('s'):
                save_count += 1
                filename = f"detection_frame_{save_count:04d}.jpg"
                cv2.imwrite(filename, frame)
                print(f"💾 Frame saved: {filename}")
            elif key == ord('r'):
                print("\n📋 Recent License Plate Detections:")
                if detector.recent_plates:
                    for i, detection in enumerate(reversed(list(detector.recent_plates))):
                        timestamp = time.strftime('%H:%M:%S', time.localtime(detection['timestamp']))
                        print(f"   {i+1:2d}. {detection['text']:12s} | "
                              f"Plate: {detection['confidence']:.2f} | "
                              f"OCR: {detection['ocr_confidence']:.2f} | "
                              f"{timestamp}")
                else:
                    print("   No detections yet")
            elif key == ord('c'):
                detector.recent_plates.clear()
                detector.detection_history.clear()
                print("🗑️  Detection history cleared")
    
    except KeyboardInterrupt:
        print("\n⏹️  Interrupted by user")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Webcam released and windows closed")
        
        print(f"\n📊 Session Statistics:")
        print(f"   Total frames processed: {frame_count}")
        print(f"   Total detections: {len(detector.recent_plates)}")
        print(f"   Frames saved: {save_count}")

if __name__ == "__main__":
    main()

C:\Users\minhk\anaconda3\Lib\site-packages\yolov5\utils\general.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg


✅ YOLOv5 package available
🚗 License Plate Detection and OCR System
🔍 Searching in: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam
🎯 Found OCR model: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam\datasetocr\models\ocr_character_recognition\weights\ocr_character_detection_best.pt
🎯 Found plate model: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt
🔍 Searching in: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset
🎯 Found plate model: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/dataset\models\red_plate_memory_opt\weights\best.pt
🔍 Searching in: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr
🎯 Found OCR model: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam/datasetocr\models\ocr_character_recognition\weights\ocr_character_detection_best.pt

📁 Model Status:
   License Plate Model: C:/Users/minhk/Downloads/code thuê/AI/NguyenLam\dataset\models\red_plate_memory_opt\weights\best.pt
   OCR Model: C:/Users/minhk/Downloa

YOLOv5  2025-6-22 Python-3.12.4 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)



❌ torch.load failed: cannot instantiate 'PosixPath' on your system
❌ yolov5.load failed: Invalid CUDA '--device cuda' requested, use '--device cpu' or pass valid CUDA device(s)
❌ Ultralytics YOLO failed: cannot instantiate 'PosixPath' on your system
🔄 Trying manual YOLOv5 model loading...
❌ Manual load failed: cannot instantiate 'PosixPath' on your system
⚠️ YOLOv5 wrapper loading failed: Failed to load YOLOv5 model with all methods


WARNING  DetectMultiBackend failed: cannot instantiate 'PosixPath' on your system


⚠️ Direct YOLOv5 loading failed: cannot instantiate 'PosixPath' on your system
❌ Direct torch loading failed: cannot instantiate 'PosixPath' on your system
❌ All OCR model loading methods failed

📷 Initializing webcam...



KeyboardInterrupt

